In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
%%capture
! pip install duckdb scikit-survival

In [3]:
import duckdb
from collections import defaultdict
from sksurv.metrics import concordance_index_censored
def c_index(y, p):
    name_event, name_time = y.dtype.names
    result = concordance_index_censored(y[name_event], y[name_time], p)
    return result[0]

#### Load data

In [4]:
path = '/kaggle/input/churn/'

In [5]:
clients = pd.read_csv(path + 'clients.csv')
mcc_codes = pd.read_csv(path + 'mcc_codes.csv')
report_dates = pd.read_csv(path + 'report_dates.csv')
train = pd.read_csv(path + 'train.csv')
currency = pd.read_csv(path + 'currency_rk.csv')
sample_submission = pd.read_csv(path + 'sample_submit_naive.csv')
transactions = pd.read_csv(path + 'transactions.csv')

In [6]:
transactions['transaction_dttm'] = pd.to_datetime(transactions['transaction_dttm'])
report_dates['report_dt'] = pd.to_datetime(report_dates['report_dt'])

In [7]:
t = duckdb.sql('''SELECT t1.*, t2.report_dt, t3.target, t3.time
                    FROM clients t1
               LEFT JOIN report_dates t2 
                      ON t1.report = t2.report
               LEFT JOIN train t3
                      ON t1.user_id = t3.user_id''').to_df()

In [8]:
df = t.copy()

#### Досчитываем поля в transactions

In [9]:
transactions['day'] = transactions['transaction_dttm'].dt.floor('d')
transactions['week'] = transactions['transaction_dttm'].to_numpy().astype('datetime64[W]')
transactions['month'] = transactions['transaction_dttm'].to_numpy().astype('datetime64[M]')
transactions['day_of_week'] = transactions['day'].dt.dayofweek
transactions['hour'] = transactions['transaction_dttm'].dt.hour

In [10]:
transactions = duckdb.sql(f'''SELECT t1.*, 
                                    t1.day - lag(t1.day) OVER(PARTITION BY t1.user_id ORDER BY t1.transaction_dttm) as days_lag, 
                                    t2.report_dt::date - t1.day as days_to_rep,
                                    lead(t1.day) OVER(PARTITION BY t1.user_id ORDER BY t1.transaction_dttm) - t1.day as days_lead, 
                                    t2.target
                               FROM transactions t1
                          LEFT JOIN t t2
                                 ON t1.user_id = t2.user_id''').to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
transactions['days_lag'] = transactions['days_lag'].dt.days
transactions['days_to_rep'] = transactions['days_to_rep'].dt.days
transactions['days_lead'] = transactions['days_lead'].dt.days
transactions['weeks_to_rep'] = transactions['days_to_rep'] // 7
transactions['months_to_rep'] = transactions['days_to_rep'] // 17

In [12]:
v = duckdb.sql('''SELECT DISTINCT user_id, target, mcc_code
                             FROM transactions''')

u = duckdb.sql('''SELECT mcc_code, count(target) as target_cnt
                    FROM v
                GROUP BY mcc_code
                  HAVING count(target) > 500''').to_df()['mcc_code'].tolist()

transactions['mcc_code_old'] = transactions['mcc_code']

transactions.loc[~transactions['mcc_code'].isin(u), 'mcc_code'] = -1 

w = duckdb.sql('''SELECT DISTINCT user_id, target, mcc_code
                             FROM transactions''')

mcc = duckdb.sql('''SELECT mcc_code, count(*) as target_cnt, avg(target) as target_mean
                      FROM w
                  GROUP BY mcc_code
                    HAVING count(target) > 500''').to_df()

In [13]:
mcc = duckdb.sql('''SELECT *, RANK() OVER(ORDER BY target_cnt desc) as cnt_rank FROM mcc''').to_df()

In [14]:
transactions = duckdb.sql('''SELECT t1.*, t2.target_cnt as mcc_cnt, t2.target_mean as mcc_target, t2.cnt_rank as mcc_cnt_rank
                               FROM transactions t1
                          LEFT JOIN mcc t2
                                 ON t1.mcc_code = t2.mcc_code''').to_df()

In [15]:
mcc.sort_values(by='target_mean', ascending=False)[:5]

,mcc_code,target_cnt,target_mean,cnt_rank
101,51,2365,0.333991,101
137,89,947,0.127962,138
106,52,2114,0.106277,107
31,26,14438,0.104902,32
85,29,3485,0.104274,86


In [16]:
transactions['transaction_amt_cat'] = (transactions['transaction_amt'] // 100).astype(int).map(lambda x: 999 if x > 2 else x)\
.map(lambda x: -999 if x < -5 else x)

#### Сборка признаков

In [17]:
feat_7 = duckdb.sql('''
                     SELECT user_id,
                            sum(transaction_amt) as tr_amt_rub,
                            count(transaction_amt) as tr_cnt_rub,
                            sum(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_rub,
                            sum(case when transaction_amt > 0 then 1 else 0 end) as tr_cnt_popoln_rub,
                            sum(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_rub,
                            sum(case when transaction_amt < 0 then 1 else 0 end) as tr_cnt_out_rub,
                            min(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_max_rub,
                            count(distinct day) as tr_cnt_days_rub,
                            count(distinct case when transaction_amt > 0 then day end) as tr_cnt_days_popoln_rub,
                            avg(days_lag) as tr_days_lag_avg_rub,
                            max(days_lag) as tr_days_lag_max_rub,
                            min(days_to_rep) - avg(days_lag) as tr_last_date_plus_avg_lag_rub,
                            min(days_to_rep) - max(days_lag) as tr_last_date_plus_max_lag_rub,
                            max(days_to_rep) as tr_first_date_rub,
                            avg(transaction_amt) as tr_amt_avg_rub,
                            avg(case when transaction_amt > 0 then transaction_amt end) as tr_amt_popoln_avg_rub,
                            avg(case when transaction_amt < 0 then transaction_amt end) as tr_amt_out_avg_rub,
                            max(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_max_rub,
                            count(distinct mcc_code_old) as tr_cnt_mcc_rub,
                            sum(case when mcc_code=12 then 1 else 0 end) as tr_cnt_mcc_12_rub,
                            sum(case when transaction_amt < 0 and transaction_amt > -100 then 1 else 0 end) as tr_low_100_rub,
                            sum(case when transaction_amt >= 20000 then 1 else 0 end) as tr_high_20000_rub
                       FROM transactions
                      WHERE currency_rk = 1
                   GROUP BY user_id''').to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
feat_6 = duckdb.sql('''
                     SELECT user_id, 
                            min(days_to_rep) as tr_last_date_155,
                            sum(transaction_amt) as tr_amt_155,
                            count(transaction_amt) as tr_cnt_155,
                            sum(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_155,
                            sum(case when transaction_amt > 0 then 1 else 0 end) as tr_cnt_popoln_155,
                            sum(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_155,
                            sum(case when transaction_amt < 0 then 1 else 0 end) as tr_cnt_out_155,
                            min(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_max_155,
                            count(distinct day) as tr_cnt_days_155,
                            count(distinct case when transaction_amt > 0 then day end) as tr_cnt_days_popoln_155,
                            avg(days_lag) as tr_days_lag_avg_155,
                            max(days_lag) as tr_days_lag_max_155,
                            min(days_to_rep) - avg(days_lag) as tr_last_date_plus_avg_lag_155,
                            min(days_to_rep) - max(days_lag) as tr_last_date_plus_max_lag_155,
                            max(days_to_rep) as tr_first_date_155,
                            avg(transaction_amt) as tr_amt_avg_155,
                            avg(case when transaction_amt > 0 then transaction_amt end) as tr_amt_popoln_avg_155,
                            avg(case when transaction_amt < 0 then transaction_amt end) as tr_amt_out_avg_155,
                            max(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_max_155,
                            sum(case when transaction_amt < 0 and transaction_amt > -100 then 1 else 0 end) as tr_low_100_155
                       FROM transactions
                      WHERE mcc_code=155
                   GROUP BY user_id''').to_df()

In [19]:
feat_5 = duckdb.sql('''
                     SELECT user_id, 
                            min(days_to_rep) as tr_last_date_0,
                            sum(transaction_amt) as tr_amt_0,
                            count(transaction_amt) as tr_cnt_0,
                            sum(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_0,
                            sum(case when transaction_amt > 0 then 1 else 0 end) as tr_cnt_popoln_0,
                            sum(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_0,
                            sum(case when transaction_amt < 0 then 1 else 0 end) as tr_cnt_out_0,
                            min(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_max_0,
                            count(distinct day) as tr_cnt_days_0,
                            count(distinct case when transaction_amt > 0 then day end) as tr_cnt_days_popoln_0,
                            avg(days_lag) as tr_days_lag_avg_0,
                            max(days_lag) as tr_days_lag_max_0,
                            min(days_to_rep) - avg(days_lag) as tr_last_date_plus_avg_lag_0,
                            min(days_to_rep) - max(days_lag) as tr_last_date_plus_max_lag_0,
                            max(days_to_rep) as tr_first_date_0,
                            avg(transaction_amt) as tr_amt_avg_0,
                            avg(case when transaction_amt > 0 then transaction_amt end) as tr_amt_popoln_avg_0,
                            avg(case when transaction_amt < 0 then transaction_amt end) as tr_amt_out_avg_0,
                            max(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_max_0,
                            sum(case when transaction_amt < 0 and transaction_amt > -100 then 1 else 0 end) as tr_low_100_0
                FROM transactions
               WHERE mcc_code=0
            GROUP BY user_id''').to_df()

In [20]:
feat_4 = duckdb.sql('''
                     SELECT user_id, 
                            min(days_to_rep) as tr_last_date_51,
                            sum(transaction_amt) as tr_amt_51,
                            count(transaction_amt) as tr_cnt_51,
                            sum(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_51,
                            sum(case when transaction_amt > 0 then 1 else 0 end) as tr_cnt_popoln_51,
                            sum(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_51,
                            sum(case when transaction_amt < 0 then 1 else 0 end) as tr_cnt_out_51,
                            min(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_max_51,
                            count(distinct day) as tr_cnt_days_51,
                            count(distinct case when transaction_amt > 0 then day end) as tr_cnt_days_popoln_51,
                            avg(days_lag) as tr_days_lag_avg_51,
                            max(days_lag) as tr_days_lag_max_51,
                            min(days_to_rep) - avg(days_lag) as tr_last_date_plus_avg_lag_51,
                            min(days_to_rep) - max(days_lag) as tr_last_date_plus_max_lag_51,
                            max(days_to_rep) as tr_first_date_51,
                            avg(transaction_amt) as tr_amt_avg_51,
                            avg(case when transaction_amt > 0 then transaction_amt end) as tr_amt_popoln_avg_51,
                            avg(case when transaction_amt < 0 then transaction_amt end) as tr_amt_out_avg_51,
                            max(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_max_51,
                            sum(case when transaction_amt < 0 and transaction_amt > -100 then 1 else 0 end) as tr_low_100_51
                       FROM transactions
                      WHERE mcc_code=51
                   GROUP BY user_id''').to_df()

In [21]:
feat_3 = duckdb.sql('''
                     SELECT user_id, 
                            min(days_to_rep) as tr_last_date_12,
                            sum(transaction_amt) as tr_amt_12,
                            count(transaction_amt) as tr_cnt_12,
                            sum(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_12,
                            sum(case when transaction_amt > 0 then 1 else 0 end) as tr_cnt_popoln_12,
                            sum(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_12,
                            sum(case when transaction_amt < 0 then 1 else 0 end) as tr_cnt_out_12,
                            min(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_max_12,
                            count(distinct day) as tr_cnt_days_12,
                            count(distinct case when transaction_amt > 0 then day end) as tr_cnt_days_popoln_12,
                            avg(days_lag) as tr_days_lag_avg_12,
                            max(days_lag) as tr_days_lag_max_12,
                            min(days_to_rep) - avg(days_lag) as tr_last_date_plus_avg_lag_12,
                            min(days_to_rep) - max(days_lag) as tr_last_date_plus_max_lag_12,
                            max(days_to_rep) as tr_first_date_12,
                            avg(transaction_amt) as tr_amt_avg_12,
                            avg(case when transaction_amt > 0 then transaction_amt end) as tr_amt_popoln_avg_12,
                            avg(case when transaction_amt < 0 then transaction_amt end) as tr_amt_out_avg_12,
                            max(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_max_12,
                            sum(case when transaction_amt < 0 and transaction_amt > -100 then 1 else 0 end) as tr_low_100_12
                FROM transactions
               WHERE mcc_code=12
            GROUP BY user_id''').to_df()

In [22]:
feat_2 = duckdb.sql('''
                     SELECT user_id,
                            sum(transaction_amt) as tr_amt_month,
                            count(transaction_amt) as tr_cnt_month,
                            sum(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_month,
                            sum(case when transaction_amt > 0 then 1 else 0 end) as tr_cnt_popoln_month,
                            sum(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_month,
                            sum(case when transaction_amt < 0 then 1 else 0 end) as tr_cnt_out_month,
                            min(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_max_month,
                            count(distinct day) as tr_cnt_days_month,
                            count(distinct case when transaction_amt > 0 then day end) as tr_cnt_days_popoln_month,
                            avg(days_lag) as tr_days_lag_avg_month,
                            max(days_lag) as tr_days_lag_max_month,
                            min(days_to_rep) - avg(days_lag) as tr_last_date_plus_avg_lag_month,
                            min(days_to_rep) - max(days_lag) as tr_last_date_plus_max_lag_month,
                            max(days_to_rep) as tr_first_date_month,
                            avg(transaction_amt) as tr_amt_avg_month,
                            avg(case when transaction_amt > 0 then transaction_amt end) as tr_amt_popoln_avg_month,
                            avg(case when transaction_amt < 0 then transaction_amt end) as tr_amt_out_avg_month,
                            max(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_max_month,
                            count(distinct mcc_code_old) as tr_cnt_mcc_month,
                            sum(case when mcc_code=12 then 1 else 0 end) as tr_cnt_mcc_12_month,
                            sum(case when transaction_amt < 0 and transaction_amt > -100 then 1 else 0 end) as tr_low_100_month,
                            sum(case when transaction_amt >= 20000 then 1 else 0 end) as tr_high_20000_month
                       FROM transactions
                      WHERE days_to_rep < 133
                   GROUP BY user_id''').to_df()

In [23]:
feat = duckdb.sql('''SELECT user_id, 
                            min(days_to_rep) as tr_last_date, 
                            sum(transaction_amt) as tr_amt_total,
                            count(transaction_amt) as tr_cnt_total,
                            sum(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_total,
                            sum(case when transaction_amt > 0 then 1 else 0 end) as tr_cnt_popoln_total,
                            sum(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_total,
                            sum(case when transaction_amt < 0 then 1 else 0 end) as tr_cnt_out_total,
                            min(case when transaction_amt < 0 then transaction_amt else 0 end) as tr_amt_out_max_total,
                            count(distinct day) as tr_cnt_days_total,
                            count(distinct case when transaction_amt > 0 then day end) as tr_cnt_days_popoln_total,
                            avg(days_lag) as tr_days_lag_avg_total,
                            max(days_lag) as tr_days_lag_max_total,
                            min(days_to_rep) - avg(days_lag) as tr_last_date_plus_avg_lag_total,
                            min(days_to_rep) - max(days_lag) as tr_last_date_plus_max_lag_total,
                            max(days_to_rep) as tr_first_date_total,
                            avg(transaction_amt) as tr_amt_avg_total,
                            avg(case when transaction_amt > 0 then transaction_amt end) as tr_amt_popoln_avg_total,
                            avg(case when transaction_amt < 0 then transaction_amt end) as tr_amt_out_avg_total,
                            max(case when transaction_amt > 0 then transaction_amt else 0 end) as tr_amt_popoln_max_total,
                            count(distinct mcc_code_old) as tr_cnt_mcc_total,
                            avg(mcc_target) as tr_mcc_target_avg_total,
                            sum(case when mcc_code=12 then 1 else 0 end) as tr_cnt_mcc_12_total,
                            sum(case when mcc_code=51 then 1 else 0 end) as tr_cnt_mcc_51_total,
                            sum(case when mcc_code=0 then 1 else 0 end) as tr_cnt_mcc_0_total,
                            sum(case when mcc_code=155 then 1 else 0 end) as tr_cnt_mcc_155_total,
                            sum(case when transaction_amt < 0 and transaction_amt > -100 then 1 else 0 end) as tr_low_100_total,
                            sum(case when transaction_amt >= 20000 then 1 else 0 end) as tr_high_20000_total,
                            min(currency_rk) as tr_min_currecncy_total
                       FROM transactions
                   GROUP BY user_id''').to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [24]:
t = df.copy()

In [25]:
t = duckdb.sql('''SELECT t1.*, t2.*, t3.*, t4.*, t5.*, t8.*, t6.*, t7.*
                    FROM t t1
               LEFT JOIN feat t2
                      ON t1.user_id = t2.user_id
               LEFT JOIN feat_2 t3
                      ON t1.user_id = t3.user_id
               LEFT JOIN feat_3 t4
                      ON t1.user_id = t4.user_id
               LEFT JOIN feat_4 t5
                      ON t1.user_id = t5.user_id
               LEFT JOIN feat_5 t6
                      ON t1.user_id = t6.user_id
               LEFT JOIN feat_6 t7
                      ON t1.user_id = t7.user_id
               LEFT JOIN feat_7 t8
                      ON t1.user_id = t8.user_id
               ''').to_df()

In [26]:
t = t.drop(columns = ['user_id_1', 'user_id_2', 'user_id_3', 'user_id_4', 'user_id_5', 'user_id_6', 'user_id_7'])

In [27]:
t['employee_count_nm'] = t['employee_count_nm'].map({'БОЛЕЕ 1001': 1000, 'ОТ 101 ДО 500': 100, 'ОТ 51 ДО 100': 50, 
                                                     'ОТ 501 ДО 1000': 750, 'ОТ 11 ДО 50': 30, 'ДО 10': 1, 
                                                     'БОЛЕЕ 500': 500, 'ОТ 11 ДО 30': 20, 'ОТ 31 ДО 50': 40}).fillna(-1)

In [28]:
tmp = transactions.copy()
tmp['hour'] = tmp['transaction_dttm'].dt.hour
pivot_table = tmp.pivot_table(
    index='user_id',
    columns='hour',
    values='transaction_amt',
    aggfunc=['count', 'sum']
).fillna(0)
pivot_table.columns = [f'hour_{x[0]}_{x[1]}' for x in pivot_table.columns]

count_cols = [x for x in pivot_table.columns if 'count' in x]
pivot_table['sum'] = pivot_table[count_cols].sum(axis=1)
for col in count_cols:
    pivot_table[f'{col}_norm'] = pivot_table[col] / pivot_table['sum']
pivot_table.drop('sum', axis=1, inplace=True)

pivot_table = pivot_table[[col for col in pivot_table.columns if col.find('norm') >= 0 and col.find('count') >= 0]]

t = t.merge(pivot_table, how='left', left_on='user_id', right_index=True)

In [29]:
tmp = transactions.copy()
pivot_table = tmp.pivot_table(
    index='user_id',
    columns='transaction_amt_cat',
    values='transaction_amt',
    aggfunc=['count', 'sum']
).fillna(0)
pivot_table.columns = [f'tr_amt_{x[0]}_{x[1]}' for x in pivot_table.columns]

count_cols = [x for x in pivot_table.columns if 'count' in x]
pivot_table['sum'] = pivot_table[count_cols].sum(axis=1)
for col in count_cols:
    pivot_table[f'{col}_norm'] = pivot_table[col] / pivot_table['sum']
pivot_table.drop('sum', axis=1, inplace=True)

pivot_table = pivot_table[[col for col in pivot_table.columns if col.find('norm') >= 0 and col.find('count') >= 0]]

t = t.merge(pivot_table, how='left', left_on='user_id', right_index=True)

In [30]:
tmp = transactions.copy()
pivot_table = tmp.pivot_table(
    index='user_id',
    columns='weeks_to_rep',
    values='transaction_amt',
    aggfunc=['count', 'sum']
).fillna(0)
pivot_table.columns = [f'tr_week_{x[0]}_{x[1]}' for x in pivot_table.columns]

count_cols = [x for x in pivot_table.columns if 'count' in x]
pivot_table['sum'] = pivot_table[count_cols].sum(axis=1)
for col in count_cols:
    pivot_table[f'{col}_norm'] = pivot_table[col] / pivot_table['sum']
pivot_table.drop('sum', axis=1, inplace=True)

pivot_table = pivot_table[[col for col in pivot_table.columns if col.find('norm') < 0 and col.find('count') >= 0]]

t = t.merge(pivot_table, how='left', left_on='user_id', right_index=True)

In [31]:
tmp = transactions.copy()
pivot_table = tmp.pivot_table(
    index='user_id',
    columns='months_to_rep',
    values='transaction_amt',
    aggfunc=['count', 'sum', 'mean']
).fillna(0)
pivot_table.columns = [f'tr_month_{x[0]}_{x[1]}' for x in pivot_table.columns]

count_cols = [x for x in pivot_table.columns if 'count' in x]
pivot_table['sum'] = pivot_table[count_cols].sum(axis=1)
for col in count_cols:
    pivot_table[f'{col}_norm'] = pivot_table[col] / pivot_table['sum']
pivot_table.drop('sum', axis=1, inplace=True)

pivot_table = pivot_table[[col for col in pivot_table.columns if col.find('norm') >= 0 or col.find('count') < 0]]

t = t.merge(pivot_table, how='left', left_on='user_id', right_index=True)

In [32]:
tmp = transactions[(transactions['mcc_code'] >= 0) & (transactions['mcc_code'].isin(mcc[mcc['cnt_rank'] <= 35]['mcc_code']))].copy()
pivot_table = tmp.pivot_table(
    index='user_id',
    columns='mcc_code',
    values='transaction_amt',
    aggfunc=['count', 'sum']
).fillna(0)
pivot_table.columns = [f'tr_mcc_{x[0]}_{x[1]}' for x in pivot_table.columns]

count_cols = [x for x in pivot_table.columns if 'count' in x]
pivot_table['sum'] = pivot_table[count_cols].sum(axis=1)
for col in count_cols:
    pivot_table[f'{col}_norm'] = pivot_table[col] / pivot_table['sum']
pivot_table.drop('sum', axis=1, inplace=True)

pivot_table = pivot_table[[col for col in pivot_table.columns if col.find('norm') >= 0 or col.find('count') < 0]]

t = t.merge(pivot_table, how='left', left_on='user_id', right_index=True)

In [33]:
t['report_cat'] = t['report'].astype('category')

In [34]:
y = t[['target', 'time']]
not_features = ['report_dt', 'target', 'time']
features = [col for col in t.columns if col not in not_features]
features

['user_id',
 'report',
 'employee_count_nm',
 'bankemplstatus',
 'customer_age',
 'tr_last_date',
 'tr_amt_total',
 'tr_cnt_total',
 'tr_amt_popoln_total',
 'tr_cnt_popoln_total',
 'tr_amt_out_total',
 'tr_cnt_out_total',
 'tr_amt_out_max_total',
 'tr_cnt_days_total',
 'tr_cnt_days_popoln_total',
 'tr_days_lag_avg_total',
 'tr_days_lag_max_total',
 'tr_last_date_plus_avg_lag_total',
 'tr_last_date_plus_max_lag_total',
 'tr_first_date_total',
 'tr_amt_avg_total',
 'tr_amt_popoln_avg_total',
 'tr_amt_out_avg_total',
 'tr_amt_popoln_max_total',
 'tr_cnt_mcc_total',
 'tr_mcc_target_avg_total',
 'tr_cnt_mcc_12_total',
 'tr_cnt_mcc_51_total',
 'tr_cnt_mcc_0_total',
 'tr_cnt_mcc_155_total',
 'tr_low_100_total',
 'tr_high_20000_total',
 'tr_min_currecncy_total',
 'tr_amt_month',
 'tr_cnt_month',
 'tr_amt_popoln_month',
 'tr_cnt_popoln_month',
 'tr_amt_out_month',
 'tr_cnt_out_month',
 'tr_amt_out_max_month',
 'tr_cnt_days_month',
 'tr_cnt_days_popoln_month',
 'tr_days_lag_avg_month',
 'tr_days

In [35]:
len(features)

320

#### Обучение моделей на кросс валидации

In [36]:
target_enc_cols = ['tr_mcc_target_avg_total']
monotone_constraints = [1 if col in target_enc_cols else 0 for col in features]

In [37]:
cat_features = [features.index('report_cat')]

In [38]:
from catboost import CatBoostClassifier

params_2 = {
    'n_estimators': 5000,
    'depth': 7,
    'learning_rate': 0.01,
    'eval_metric': 'AUC',
    'cat_features': cat_features,
    'early_stopping_rounds': 500,
    'metric_period': 500,
    'task_type': 'GPU', 
    'devices': '0:1'
}

In [39]:
from xgboost import XGBClassifier

params_4 = {
    'reg_lambda': 2.954282453141972,
    'reg_alpha': 2.714226004554789,
    'subsample': 0.8747340624732572,
    'colsample_bytree': 0.28932012021539283,
    'gamma': 2.2132144067786714,
    'min_child_weight': 18,
    'max_depth': 7,
    'learning_rate': 0.004519460989612957,
    'tree_method': 'hist',
    'n_estimators': 5000,
    'monotone_constraints': tuple(monotone_constraints),
    'enable_categorical': True,
    'early_stopping_rounds': 500,
    'device': 'cuda'
}

In [40]:
from xgboost import XGBClassifier

params_6 = {
    'reg_lambda': 2.954282453141972,
    'reg_alpha': 2.714226004554789,
    'subsample': 0.8747340624732572,
    'colsample_bytree': 0.28932012021539283,
    'gamma': 2.2132144067786714,
    'min_child_weight': 18,
    'max_depth': 7,
    'learning_rate': 0.004519460989612957,
    'tree_method': 'hist',
    'n_estimators': 5000,
    'monotone_constraints': tuple(monotone_constraints),
    'enable_categorical': True,
    'early_stopping_rounds': 500,
    'device': 'cuda'
}

In [41]:
import lightgbm as lgb

params_7 = {
    'n_estimators'     : 5000,
    'objective'        : 'binary',
    'verbose'          : -1,
    'learning_rate'    : 0.005, 
    'colsample_bytree' : 0.5099540080762465, 
    'colsample_bynode' : 0.4575886155554022, 
    'lambda_l1'        : 3.2150774310219994, 
    'lambda_l2'        : 6.026390839987788, 
    'min_data_in_leaf' : 86, 
    'max_depth'        : 6, 
    'num_leaves'       : 174, 
    'max_bin'          : 255,
    'monotone_constraints': monotone_constraints,
    'device'           : 'gpu',
    'early_stopping_rounds': 500
}

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [42]:
from xgboost import XGBRegressor

params_9 = {
    'objective': 'survival:cox',
    'reg_lambda': 2.954282453141972,
    'reg_alpha': 2.714226004554789,
    'subsample': 0.8747340624732572,
    'colsample_bytree': 0.28932012021539283,
    'gamma': 2.2132144067786714,
    'min_child_weight': 18,
    'max_depth': 7,
    'learning_rate': 0.004519460989612957,
    'tree_method': 'hist',
    'n_estimators': 5000,
    'monotone_constraints': tuple(monotone_constraints),
    'enable_categorical': True,
    'early_stopping_rounds': 500,
    'device': 'cuda'
}

In [43]:
import xgboost as xgb

params_10 = {
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'aft_loss_distribution': 'normal',
    'aft_loss_distribution_scale': 1.20,
    'reg_lambda': 2.954282453141972,
    'reg_alpha': 2.714226004554789,
    'subsample': 0.8747340624732572,
    'colsample_bytree': 0.28932012021539283,
    'gamma': 2.2132144067786714,
    'min_child_weight': 18,
    'max_depth': 7,
    'learning_rate': 0.004519460989612957,
    'tree_method': 'hist',
    #'n_estimators': 5000,
    'monotone_constraints': tuple(monotone_constraints),
    'enable_categorical': True,
    #'early_stopping_rounds': 500,
    'device': 'cuda'
}

In [44]:
p_train = defaultdict(list)
p_valid = defaultdict(list)
valid_loss = defaultdict(list)
n_iter = defaultdict(list)
for i in tqdm(range(5)):
    train_mask = t['target'].notnull() & (t['user_id'] % 5 != i)
    valid_mask = t['target'].notnull() & (t['user_id'] % 5 == i)
    X_train = t.loc[train_mask, features]
    y_train = np.array(list(y.loc[train_mask].itertuples(index = False, name = None)), dtype=[('target', np.bool_), ('time', 'f8')])
    X_valid = t.loc[valid_mask, features]
    y_valid = np.array(list(y.loc[valid_mask].itertuples(index = False, name = None)), dtype=[('target', np.bool_), ('time', 'f8')])
    y_train_cbt = np.where(y.loc[train_mask]['target'], y.loc[train_mask]['time'], -y.loc[train_mask]['time'])
    y_valid_cbt = np.where(y.loc[valid_mask]['target'], y.loc[valid_mask]['time'], -y.loc[valid_mask]['time'])
    y_train_cbt_2 = y.loc[train_mask].copy()
    y_train_cbt_2['y_lower'] = np.where(y_train_cbt_2['target'], y_train_cbt_2['time'], 91)
    y_train_cbt_2['y_upper'] = np.where(y_train_cbt_2['target'], y_train_cbt_2['time'], +np.inf)
    y_train_cbt_2 = y_train_cbt_2[['y_lower', 'y_upper']]
    y_valid_cbt_2 = y.loc[valid_mask].copy()
    y_valid_cbt_2['y_lower'] = np.where(y_valid_cbt_2['target'], y_valid_cbt_2['time'], 91)
    y_valid_cbt_2['y_upper'] = np.where(y_valid_cbt_2['target'], y_valid_cbt_2['time'], +np.inf)
    y_valid_cbt_2 = y_valid_cbt_2[['y_lower', 'y_upper']]
    y_train_lgb = y.loc[train_mask]['target']
    y_valid_lgb = y.loc[valid_mask]['target']
    y_2 = y.copy()
    y_2['target_2'] = y_2['target']
    y_2.loc[(y_2['target'] == 1), 'target_2'] = 2
    y_2.loc[(y_2['target'] == 0) & (y_2['time'] < 45), 'target_2'] = 1
    y_train_lgb_2 = y_2.loc[train_mask]['target_2']
    y_valid_lgb_2 = y_2.loc[valid_mask]['target_2']
    y_train_lgb_3 = y.loc[train_mask]['time'] + 60*(1-y.loc[train_mask]['target'])
    y_valid_lgb_3 = y.loc[valid_mask]['time'] + 60*(1-y.loc[valid_mask]['target'])
    y_train_lgb_4 = (y.loc[train_mask]['time'] <= 80).map(int)
    y_valid_lgb_4 = (y.loc[valid_mask]['time'] <= 80).map(int)
    
    
    
    model_4 = XGBClassifier(**params_4)
    model_4.fit(X_train, y_train_lgb, eval_set=[(X_valid, y_valid_lgb)], verbose=False)
    if params_4.get('early_stopping_rounds') is not None:
        print(f"Training until validation scores don't improve for {params_4.get('early_stopping_rounds')} rounds")
        print('Early stopping, best iteration is:')
        print(f"[{model_4.best_iteration}]    valid_0's binary_logloss: {model_4.best_score}")
    
    p_train['xgb'].append(model_4.predict_proba(X_train)[:, 1])
    p_valid['xgb'].append(model_4.predict_proba(X_valid)[:, 1])
    valid_loss['xgb'].append(model_4.best_score)
    n_iter['xgb'].append(model_4.best_iteration) 
    print(c_index(y_train, p_train['xgb'][-1]))
    print(c_index(y_valid, p_valid['xgb'][-1]))
    
    
    
    model_6 = XGBClassifier(**params_6)
    model_6.fit(X_train, y_train_lgb_2, eval_set=[(X_valid, y_valid_lgb_2)], verbose=False)
    if params_6.get('early_stopping_rounds') is not None:
        print(f"Training until validation scores don't improve for {params_6.get('early_stopping_rounds')} rounds")
        print('Early stopping, best iteration is:')
        print(f"[{model_6.best_iteration}]    valid_0's multi_logloss: {model_6.best_score}")
    
    p_train['xgb_multi'].append(model_6.predict_proba(X_train))
    p_valid['xgb_multi'].append(model_6.predict_proba(X_valid))
    valid_loss['xgb_multi'].append(model_6.best_score)
    n_iter['xgb_multi'].append(model_6.best_iteration) 
    
    
    
    model_7 = lgb.LGBMClassifier(**params_7, random_state=100)
    model_7.fit(X_train, y_train_lgb_4, eval_set=(X_valid, y_valid_lgb_4))
    if params_7.get('early_stopping_rounds') is not None:
        print(f"Training until validation scores don't improve for {params_7.get('early_stopping_rounds')} rounds")
        print('Early stopping, best iteration is:')
        print(f"[{model_7.best_iteration_}]    valid_0's loss: {model_7.best_score_['valid_0']}")
        
    p_train['lgb_2'].append(model_7.predict_proba(X_train)[:, 1])
    p_valid['lgb_2'].append(model_7.predict_proba(X_valid)[:, 1])
    valid_loss['lgb_2'].append(model_7.best_score_['valid_0'])
    n_iter['lgb_2'].append(model_7.best_iteration_)
    print(c_index(y_train, p_train['lgb_2'][-1]))
    print(c_index(y_valid, p_valid['lgb_2'][-1]))
    
        
    
    #model_9 = XGBRegressor(**params_9)
    #model_9.fit(X_train, y_train_cbt, eval_set=[(X_valid, y_valid_cbt)], verbose=False)
    #if params_9.get('early_stopping_rounds') is not None:
    #    print(f"Training until validation scores don't improve for {params_9.get('early_stopping_rounds')} rounds")
    #    print('Early stopping, best iteration is:')
    #    print(f"[{model_9.best_iteration}]    valid_0's loss: {model_9.best_score}")
    
    #p_train['xgb_cox'].append(model_9.predict(X_train))
    #p_valid['xgb_cox'].append(model_9.predict(X_valid))
    #valid_loss['xgb_cox'].append(model_9.best_score)
    #n_iter['xgb_cox'].append(model_9.best_iteration) 
    #print(c_index(y_train, p_train['xgb_cox'][-1]))
    #print(c_index(y_valid, p_valid['xgb_cox'][-1]))
    
    
    
    #dtrain = xgb.DMatrix(X_train)
    #dtrain.set_float_info('label_lower_bound', y_train_cbt_2['y_lower'])
    #dtrain.set_float_info('label_upper_bound', y_train_cbt_2['y_upper'])
    #dvalid = xgb.DMatrix(X_valid)
    #dvalid.set_float_info('label_lower_bound', y_valid_cbt_2['y_lower'])
    #dvalid.set_float_info('label_upper_bound', y_valid_cbt_2['y_upper'])

    #model_10 = xgb.train(params_10, dtrain, num_boost_round=5000, early_stopping_rounds=500, verbose_eval=False,
    #                evals=[(dvalid, 'valid')])

    #print(f"[{model_10.best_iteration}]    valid_0's loss: {model_10.best_score}")

    #p_train['xgb_aft'].append(model_10.predict(dtrain, iteration_range=(0,model_10.best_iteration)))
    #p_valid['xgb_aft'].append(model_10.predict(dvalid, iteration_range=(0,model_10.best_iteration)))
    #valid_loss['xgb_aft'].append(model_10.best_score)
    #n_iter['xgb_aft'].append(model_10.best_iteration) 
    #print(c_index(y_train, p_train['xgb_aft'][-1]))
    #print(c_index(y_valid, p_valid['xgb_aft'][-1]))
    
    
    
    model_2 = CatBoostClassifier(**params_2)
    model_2.fit(X_train, y_train_lgb, eval_set=(X_valid, y_valid_lgb))
    if params_2.get('early_stopping_rounds') is not None:
        print(f"Training until validation scores don't improve for {params_2.get('early_stopping_rounds')} rounds")
        print('Early stopping, best iteration is:')
        print(f"[{model_2.best_iteration_}]    valid_0's binary_logloss: {model_2.best_score_}")
        
    p_train['cat'].append(model_2.predict_proba(X_train)[:, 1])
    p_valid['cat'].append(model_2.predict_proba(X_valid)[:, 1])
    valid_loss['cat'].append(model_2.best_score_)
    n_iter['cat'].append(model_2.best_iteration_)
    print(c_index(y_train, p_train['cat'][-1]))
    print(c_index(y_valid, p_valid['cat'][-1]))
    

  0%|          | 0/5 [00:00<?, ?it/s]

Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[3174]    valid_0's binary_logloss: 0.24513332420748826


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [20:31:21] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


0.9120124935780548
0.7815923620813752
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[3023]    valid_0's multi_logloss: 0.38962079009394257


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


...

0.8776129308554296
0.779251395417602
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[3044]    valid_0's binary_logloss: 0.23947152861216317
0.9117303056887054
0.7831731263707352
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[4176]    valid_0's multi_logloss: 0.3771466228257635
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[2289]    valid_0's loss: OrderedDict([('binary_logloss', 0.4758872520455349)])
0.7718818994354747
0.7216489348684922
0:	test: 0.6560220	best: 0.6560220 (0)	total: 32ms	remaining: 2m 39s
500:	test: 0.7715079	best: 0.7715079 (500)	total: 14.2s	remaining: 2m 7s
1000:	test: 0.7763521	best: 0.7763650 (999)	total: 28.6s	remaining: 1m 54s
1500:	test: 0.7779423	best: 0.7779423 (1500)	total: 43.3s	remaining: 1m 40s
2000:	test: 0.7789743	best: 0.7789835 (1996)	total: 57.9s	remaining: 1m 26s
2500:	test: 0.7798244	best: 0.7799873

 80%|████████  | 4/5 [36:54<09:00, 540.03s/it]

0.8964165476666618
0.7806657201223146
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[2471]    valid_0's binary_logloss: 0.2463550038874677
0.8980055265669813
0.7734085688463588
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[4236]    valid_0's multi_logloss: 0.39250465331793877
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[3605]    valid_0's loss: OrderedDict([('binary_logloss', 0.48183710245659717)])
0.789995626711855
0.712824155773613
0:	test: 0.6537481	best: 0.6537481 (0)	total: 32.4ms	remaining: 2m 41s
500:	test: 0.7630348	best: 0.7630348 (500)	total: 14.6s	remaining: 2m 10s
1000:	test: 0.7667739	best: 0.7668019 (998)	total: 29.2s	remaining: 1m 56s
1500:	test: 0.7691048	best: 0.7691312 (1496)	total: 43.5s	remaining: 1m 41s
2000:	test: 0.7700667	best: 0.7700667 (2000)	total: 57.9s	remaining: 1m 26s
2500:	test: 0.7706015	best: 0.7706

100%|██████████| 5/5 [46:29<00:00, 557.86s/it]

0.9127700259232739
0.7713581695926693


#### Blending / взвешивание на кросс валидации

In [45]:
 def objective():
    res = pd.DataFrame(columns=['train_score', 'valid_score'])
    for i in range(5):
        train_mask = ((t['target'].notnull()) & (t['user_id'] % 5 != i))
        valid_mask = ((t['target'].notnull()) & (t['user_id'] % 5 == i))
        X_train = t.loc[train_mask, features]
        y_train = np.array(list(y.loc[train_mask].itertuples(index = False, name = None)), dtype=[('target', np.bool_), ('time', 'f8')])
        X_valid = t.loc[valid_mask, features]
        y_valid = np.array(list(y.loc[valid_mask].itertuples(index = False, name = None)), dtype=[('target', np.bool_), ('time', 'f8')])

        #a = trial.suggest_float('a', 0.0, 0.1)
        #b = trial.suggest_float('b', 0.0, 2.0)
        #c = trial.suggest_float('c', 0.0, 2.0)
        #d = trial.suggest_float('d', -0.2, 0.0)
        a = 0.03
        b = 1.6
        c = 0.0
        d = -0.08

        q_train = a*p_train['lgb_2'][i] + p_train['xgb'][i] + c*p_train['cat'][i] + b*p_train['xgb_multi'][i][:, 2] + d*p_train['xgb_multi'][i][:, 0]
        q_valid = a*p_valid['lgb_2'][i] + p_valid['xgb'][i] + c*p_valid['cat'][i] + b*p_valid['xgb_multi'][i][:, 2] + d*p_valid['xgb_multi'][i][:, 0]

        res.loc[i] = [c_index(y_train, q_train), c_index(y_valid, q_valid)]
        
    return res.mean()['valid_score']

In [46]:
objective()

0.7828618127416399

In [47]:
#import optuna
#study = optuna.create_study(direction='maximize', study_name='Blend')
#study.optimize(objective, n_trials=100, show_progress_bar=False, n_jobs=-1)

In [48]:
#study.best_params

#### Обучение на всем train

In [49]:
sum(n_iter['xgb']) / len(n_iter['xgb'])

3090.2

In [50]:
p_train_2 = defaultdict(list)
p_valid_2 = defaultdict(list)
train_mask = t['target'].notnull() & (t['user_id'] % 5 != -1)
valid_mask = t['target'].notnull() & (t['user_id'] % 5 == 0)
X_train = t.loc[train_mask, features]
y_train = np.array(list(y.loc[train_mask].itertuples(index = False, name = None)), dtype=[('target', np.bool_), ('time', 'f8')])
X_valid = t.loc[valid_mask, features]
y_valid = np.array(list(y.loc[valid_mask].itertuples(index = False, name = None)), dtype=[('target', np.bool_), ('time', 'f8')])
y_train_cbt = np.where(y.loc[train_mask]['target'], y.loc[train_mask]['time'], -y.loc[train_mask]['time'])
y_valid_cbt = np.where(y.loc[valid_mask]['target'], y.loc[valid_mask]['time'], -y.loc[valid_mask]['time'])
y_train_cbt_2 = y.loc[train_mask].copy()
y_train_cbt_2['y_lower'] = np.where(y_train_cbt_2['target'], y_train_cbt_2['time'], 91)
y_train_cbt_2['y_upper'] = np.where(y_train_cbt_2['target'], y_train_cbt_2['time'], +np.inf)
y_train_cbt_2 = y_train_cbt_2[['y_lower', 'y_upper']]
y_valid_cbt_2 = y.loc[valid_mask].copy()
y_valid_cbt_2['y_lower'] = np.where(y_valid_cbt_2['target'], y_valid_cbt_2['time'], 91)
y_valid_cbt_2['y_upper'] = np.where(y_valid_cbt_2['target'], y_valid_cbt_2['time'], +np.inf)
y_valid_cbt_2 = y_valid_cbt_2[['y_lower', 'y_upper']]
y_train_lgb = y.loc[train_mask]['target']
y_valid_lgb = y.loc[valid_mask]['target']
y_2 = y.copy()
y_2['target_2'] = y_2['target']
y_2.loc[(y_2['target'] == 1), 'target_2'] = 2
y_2.loc[(y_2['target'] == 0) & (y_2['time'] < 45), 'target_2'] = 1
y_train_lgb_2 = y_2.loc[train_mask]['target_2']
y_valid_lgb_2 = y_2.loc[valid_mask]['target_2']
y_train_lgb_3 = y.loc[train_mask]['time'] + 60*(1-y.loc[train_mask]['target'])
y_valid_lgb_3 = y.loc[valid_mask]['time'] + 60*(1-y.loc[valid_mask]['target'])
y_train_lgb_4 = (y.loc[train_mask]['time'] <= 80).map(int)
y_valid_lgb_4 = (y.loc[valid_mask]['time'] <= 80).map(int)



params_14 = params_4.copy()
params_14.pop('early_stopping_rounds', None)
params_14['n_estimators'] = 2900
model_4 = XGBClassifier(**params_14)
model_4.fit(X_train, y_train_lgb, eval_set=[(X_valid, y_valid_lgb)], verbose=False)

p_train_2['xgb'].append(model_4.predict_proba(X_train)[:, 1])
p_valid_2['xgb'].append(model_4.predict_proba(X_valid)[:, 1])
print(c_index(y_train, p_train_2['xgb'][-1]))
print(c_index(y_valid, p_valid_2['xgb'][-1]))



params_16 = params_6.copy()
params_16.pop('early_stopping_rounds', None)
params_16['n_estimators'] = 3400
model_6 = XGBClassifier(**params_16)
model_6.fit(X_train, y_train_lgb_2, eval_set=[(X_valid, y_valid_lgb_2)], verbose=False)
p_train_2['xgb_multi'].append(model_6.predict_proba(X_train))
p_valid_2['xgb_multi'].append(model_6.predict_proba(X_valid))



#params_19 = params_9.copy()
#params_19.pop('early_stopping_rounds', None)
#params_19['n_estimators'] = 2500
#model_9 = XGBRegressor(**params_19)
#model_9.fit(X_train, y_train_cbt, eval_set=[(X_valid, y_valid_cbt)], verbose=False)

#p_train_2['xgb_cox'].append(model_9.predict(X_train))
#p_valid_2['xgb_cox'].append(model_9.predict(X_valid))
#print(c_index(y_train, p_train_2['xgb_cox'][-1]))
#print(c_index(y_valid, p_valid_2['xgb_cox'][-1]))



params_17 = params_7.copy()
params_17.pop('early_stopping_rounds', None)
params_17['n_estimators'] = 3000
model_7 = lgb.LGBMClassifier(**params_17, random_state=100)
model_7.fit(X_train, y_train_lgb_4, eval_set=(X_valid, y_valid_lgb_4))

p_train_2['lgb_2'].append(model_7.predict_proba(X_train)[:, 1])
p_valid_2['lgb_2'].append(model_7.predict_proba(X_valid)[:, 1])
print(c_index(y_train, p_train_2['lgb_2'][-1]))
print(c_index(y_valid, p_valid_2['lgb_2'][-1]))



params_12 = params_2.copy()
params_12.pop('early_stopping_rounds', None)
params_12['n_estimators'] = 3100
model_2 = CatBoostClassifier(**params_12)
model_2.fit(X_train, y_train_lgb, eval_set=(X_valid, y_valid_lgb))

p_train_2['cat'].append(model_2.predict_proba(X_train)[:, 1])
p_valid_2['cat'].append(model_2.predict_proba(X_valid)[:, 1])
print(c_index(y_train, p_train_2['cat'][-1]))
print(c_index(y_valid, p_valid_2['cat'][-1]))

0.8975431657965194
0.8972407807273804
0.7741848609854237
0.7702093230026673
0:	test: 0.6665691	best: 0.6665691 (0)	total: 31.6ms	remaining: 1m 38s
500:	test: 0.7861946	best: 0.7861946 (500)	total: 14s	remaining: 1m 12s
1000:	test: 0.8087493	best: 0.8087493 (1000)	total: 28.1s	remaining: 58.9s
1500:	test: 0.8279983	best: 0.8279983 (1500)	total: 41.9s	remaining: 44.7s
2000:	test: 0.8446789	best: 0.8446789 (2000)	total: 56s	remaining: 30.7s
2500:	test: 0.8604990	best: 0.8604990 (2500)	total: 1m 10s	remaining: 16.8s
3000:	test: 0.8747840	best: 0.8747840 (3000)	total: 1m 24s	remaining: 2.78s
3099:	test: 0.8774666	best: 0.8774666 (3099)	total: 1m 27s	remaining: 0us
bestTest = 0.877466619
bestIteration = 3099
0.8724022369821283
0.8709696854227529


#### predict and submission

In [51]:
#study.best_params

In [52]:
a = 0.03
b = 1.6
c = 0.0
d = -0.08

In [53]:
X = t.loc[:, features]
p = a*model_7.predict_proba(X)[:, 1] + model_4.predict_proba(X)[:, 1] + c*model_2.predict_proba(X)[:, 1] + b*model_6.predict_proba(X)[:, 2] \
                                                                                                         + d*model_6.predict_proba(X)[:, 0]
t['predict'] = p

In [54]:
oot_mask = t['target'].isnull()
t[['predict', 'report']][oot_mask].groupby('report').mean()

,predict
report,
1,0.158992
2,0.192558
3,0.195048
4,0.172557
5,0.179762
6,0.197858
7,0.175738
8,0.133188
9,0.113857


In [55]:
t[['user_id', 'predict']][oot_mask]

,user_id,predict
101,61,-0.045540
102,269,-0.005166
103,327,-0.044868
104,430,-0.071323
105,553,-0.014400
...,...,...
95981,186965,-0.027191
95982,547496,0.387591
95983,340116,0.358893
95986,266840,-0.018759


In [56]:
sample_submission

,user_id,predict
0,9,-3.184396
1,61,-2.623560
2,62,-2.840654
3,80,-3.269438
4,88,-2.068500
...,...,...
31995,561362,-2.438619
31996,561419,-2.438619
31997,561895,-2.437339
31998,561908,-2.437339


In [57]:
t[['user_id', 'predict']][oot_mask].sort_values(by='user_id').to_csv('submission.csv', index=False)